In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def get_month(path):
    month = [f for f in os.listdir(path) if not f.startswith('.')]
    return month

In [3]:
# Read date
def get_date(path,month): 
    path_date = path +'/' + month
    date = [f for f in os.listdir(path_date) if not f.startswith('.')]
    return date

In [4]:
# order data
def data_process(path_file):
    #read data
    order_data = pd.read_csv(path_file,header=None,sep=',',names=['order_id','driver_id','car_id','city','start_long','start_lat','end_long','end_lat','start_time','end_time','type','label'])
    order_data = order_data[order_data.label != 0]
    unique, counts = np.unique(order_data.label, return_counts=True)
    shared = list(unique[counts != 1])
    order_data = order_data[order_data.label.isin(shared)]
    return order_data
#     order_data.to_csv('/Users/van/Desktop/Didi_Research/play_data_extraction/' + month + '/' + date)

In [5]:
# gps data, merge all gps files into one file for each day
def get_gps(path):
    # input: /Users/van/Desktop/Didi_Research/gpsdatav2/ShangHai/gulf/2016/month/date
    # list all folders in that date
    path_file = path  + '/part-00000'
    gps_data = pd.read_csv(path_file,header = None, sep = ',',names = ['order_id','time','lon','lat','velocity'])
    return gps_data

In [6]:
# subset the gps data where it's a shared ride
def gps_process(order_data_id,gps_data):
    gps_data = gps_data[gps_data.order_id.isin(order_data_id)]
    return gps_data

In [14]:
def extract_shared_data(path):
    '''
    input:
        path: Where whole data is stored
        /Users/van/Desktop/Didi_Research
    output:
        for each day, one csv file contains only shared data 
    '''
    
    #location new files would be stored to

    file_store_to = '/Users/van/Desktop/Didi_Research/play_data_extraction/order/'
    gps_store_to = '/Users/van/Desktop/Didi_Research/play_data_extraction/gps/'
    path_to_orderdata = path + '/orderdatav2/ShangHai/gulf/2016'
    path_to_gpsdata = path +'/gpsdatav2/ShangHai/gulf/2016'
    # get number of month
    month = get_month(path_to_orderdata)
    
    # get number of dates for each month
    
    # for each month
    for i in month:
        #get number of days
        date = get_date(path_to_orderdata, i)
        
        # for each day
        for j in date:
            # get shared order data
            path_to_file = path_to_orderdata + '/' + i + '/' + j + '/' +'part-00000'
            data = data_process(path_to_file)
            name = i + '_' + j + '_' + 'order_shared.csv'
            data.to_csv(file_store_to + name)
            
            # get shared gps data
            path_to_file_gps = path_to_gpsdata + '/' + i + '/' + j
            gps_data = get_gps(path_to_file_gps)
            
            shared_order_id = data.order_id
            # subset shared data
            gps_data = gps_process(shared_order_id,gps_data)
            name_gps = i + '_' + j + '_' + 'gps_shared.csv'
            gps_data.to_csv(gps_store_to + name_gps)
            print ('month ' + i + ' day ' + j +' Done')

In [15]:
extract_shared_data('/Users/van/Desktop/Didi_Research')

KeyboardInterrupt: 